In [247]:
import pandas as pd
df=pd.read_csv("/Users/apple/Downloads/archive/Train.csv")

In [248]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [249]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [250]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [251]:
df.shape

(8523, 12)

In [252]:
categorical = []
for i in df.dtypes.index:
    if (df.dtypes[i] == 'object'):
        categorical.append(i)
print(categorical)

['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


In [253]:
df.dtypes['Item_Visibility'] == 'object'

False

In [254]:
weight_mean = pd.DataFrame(df.groupby('Item_Identifier')['Item_Weight'].mean())
weight_mean

,Item_Weight
Item_Identifier,
DRA12,11.600
DRA24,19.350
DRA59,8.270
DRB01,7.390
DRB13,6.115
...,...
NCZ30,6.590
NCZ41,19.850
NCZ42,10.500


In [255]:
missing = df['Item_Weight'].isnull()
missing

0       False
1       False
2       False
3       False
4       False
        ...  
8518    False
8519    False
8520    False
8521    False
8522    False
Name: Item_Weight, Length: 8523, dtype: bool

In [256]:
import numpy as np
for i,item in enumerate(df['Item_Identifier']):
    if (missing[i] == True):
        if item is weight_mean:
            df['Item_Weight'][i] = weight_mean.loc['Item_Identifier']['Item_Weight']
        else:
            df['Item_Weight'][i] = np.mean(df['Item_Weight'])

<ipython-input-256-20641628f12e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Item_Weight'][i] = np.mean(df['Item_Weight'])


In [257]:
df['Item_Weight'].isnull().sum()

0

In [258]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [259]:
df['Outlet_Size'].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [260]:
size_mode = df.pivot_table(values = 'Outlet_Size' , columns = 'Outlet_Type' , 
                            aggfunc = (lambda x:x.mode()))
size_mode

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,Small,Small,Medium,Medium


In [261]:
size_mode['Grocery Store']

Outlet_Size    Small
Name: Grocery Store, dtype: object

In [262]:
miss_size = df['Outlet_Size'].isnull()
miss_size

0       False
1       False
2       False
3        True
4       False
        ...  
8518    False
8519     True
8520    False
8521    False
8522    False
Name: Outlet_Size, Length: 8523, dtype: bool

In [263]:
def Outlet_size_replace(x):
    return size_mode[x]

In [264]:
Outlet_size_replace('Supermarket Type2')[0]

'Medium'

In [265]:
mvd = df.loc[miss_size]
nvd = df.loc[miss_size != True]

In [266]:
mvd['Outlet_Size'] = mvd['Outlet_Type'].apply(Outlet_size_replace)

<ipython-input-266-18644fe7db36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mvd['Outlet_Size'] = mvd['Outlet_Type'].apply(Outlet_size_replace)


In [267]:
df2 = pd.concat((mvd,nvd),ignore_index=True)

In [268]:
df2.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [269]:
df2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
1,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986
2,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350
3,NCD06,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080
4,FDE51,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,Small,Tier 3,Grocery Store,178.4344


In [270]:
df2[df2['Item_Visibility'] == 0]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDX07,19.200000,Regular,0.0,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
6,FDO23,17.850000,Low Fat,0.0,Breads,93.1436,OUT045,2002,Small,Tier 2,Supermarket Type1,2174.5028
8,NCP05,19.600000,Low Fat,0.0,Health and Hygiene,153.3024,OUT045,2002,Small,Tier 2,Supermarket Type1,2428.8384
17,FDH35,18.250000,Low Fat,0.0,Starchy Foods,164.7526,OUT045,2002,Small,Tier 2,Supermarket Type1,4604.6728
20,FDV25,5.905000,Low Fat,0.0,Canned,222.5456,OUT045,2002,Small,Tier 2,Supermarket Type1,5305.0944
...,...,...,...,...,...,...,...,...,...,...,...,...
8462,FDT48,12.857645,Low Fat,0.0,Baking Goods,196.5084,OUT027,1985,Medium,Tier 3,Supermarket Type3,793.6336
8466,FDN08,7.720000,Regular,0.0,Fruits and Vegetables,117.7466,OUT018,2009,Medium,Tier 3,Supermarket Type2,1296.3126
8486,FDV31,9.800000,LF,0.0,Fruits and Vegetables,175.2370,OUT049,1999,Medium,Tier 1,Supermarket Type1,3881.6140
8493,FDQ58,12.857645,Low Fat,0.0,Snack Foods,154.5340,OUT019,1985,Small,Tier 1,Grocery Store,459.4020


In [271]:
df2['Item_Visibility'].mean()

0.06613202877895145

In [272]:
df2['Item_Visibility'] = df2['Item_Visibility'].replace(0,df2['Item_Visibility'].mean())

In [273]:
df2[df2['Item_Visibility'] == 0]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


In [274]:
df2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDX07,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
1,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986
2,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350
3,NCD06,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080
4,FDE51,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,Small,Tier 3,Grocery Store,178.4344


In [275]:
df2['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [276]:
df2['Item_Fat_Content'] = df2['Item_Fat_Content'].replace({'LF' : 'Low Fat' , 
                                                           'low fat' : 'Low Fat' , 
                                                           'reg' : 'Regular'})

In [277]:
df2['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [278]:
df2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDX07,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
1,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986
2,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350
3,NCD06,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080
4,FDE51,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,Small,Tier 3,Grocery Store,178.4344


In [279]:
def convert_item_identifier(x):
    y=x[:2]
    return y

In [280]:
df2["Item_Identifier"]=df2["Item_Identifier"].apply(convert_item_identifier)

In [281]:
df2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FD,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
1,FD,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986
2,FD,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350
3,NC,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080
4,FD,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,Small,Tier 3,Grocery Store,178.4344


In [282]:
df2['Item_Identifier'].unique()

array(['FD', 'NC', 'DR'], dtype=object)

In [283]:
df2['Item_Identifier'] = df2['Item_Identifier'].replace({'FD':'Food', 
                                                         'NC':'Non-Consumable', 
                                                         'DR':'Drink'})

In [284]:
df2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,Food,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
1,Food,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986
2,Food,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350
3,Non-Consumable,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080
4,Food,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,Small,Tier 3,Grocery Store,178.4344


In [285]:
df2.loc[df2['Item_Identifier'] == 'Non-Consumable' , 'Item_Fat_Content'] = 'Non-Edible'

In [286]:
df2["Item_Fat_Content"].unique()

array(['Regular', 'Non-Edible', 'Low Fat'], dtype=object)

In [287]:
df2['Item_Fat_Content'].value_counts()

Low Fat       3918
Regular       3006
Non-Edible    1599
Name: Item_Fat_Content, dtype: int64

In [288]:
df2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,Food,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
1,Food,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986
2,Food,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350
3,Non-Consumable,13.000,Non-Edible,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080
4,Food,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,Small,Tier 3,Grocery Store,178.4344


In [289]:
df2['Years'] = 2020 - df2['Outlet_Establishment_Year']
df2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Years
0,Food,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,22
1,Food,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986,18
2,Food,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350,13
3,Non-Consumable,13.000,Non-Edible,0.099887,Household,45.9060,OUT017,2007,Small,Tier 2,Supermarket Type1,838.9080,13
4,Food,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,Small,Tier 3,Grocery Store,178.4344,22


In [290]:
df2['Item_Outlet_Sales'] = np.log(1+df2['Item_Outlet_Sales'])

In [291]:
df3 = df2.drop(['Outlet_Identifier','Outlet_Establishment_Year'],axis=1)

In [292]:
df3.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Years
0,Food,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,Small,Tier 3,Grocery Store,6.597664,22
1,Food,16.200,Regular,0.016687,Frozen Foods,96.9726,Small,Tier 2,Supermarket Type1,6.982490,18
2,Food,19.200,Regular,0.094450,Frozen Foods,187.8214,Small,Tier 2,Supermarket Type1,8.457769,13
3,Non-Consumable,13.000,Non-Edible,0.099887,Household,45.9060,Small,Tier 2,Supermarket Type1,6.733292,13
4,Food,5.925,Regular,0.161467,Dairy,45.5086,Small,Tier 3,Grocery Store,5.189810,22


In [293]:
df3["Outlet_Size"].unique()

array(['Small', 'Medium', 'High'], dtype=object)

In [294]:
df3["Item_Type"].unique()

array(['Fruits and Vegetables', 'Frozen Foods', 'Household', 'Dairy',
       'Breads', 'Health and Hygiene', 'Canned', 'Meat', 'Snack Foods',
       'Soft Drinks', 'Starchy Foods', 'Baking Goods', 'Others',
       'Breakfast', 'Hard Drinks', 'Seafood'], dtype=object)

In [295]:
Item_count = df3['Item_Type'].value_counts()
Item_count_less = Item_count[Item_count <= 251]
Item_count_less

Breads           251
Hard Drinks      214
Others           169
Starchy Foods    148
Breakfast        110
Seafood           64
Name: Item_Type, dtype: int64

In [296]:
def others(x):
    if x in Item_count_less:
        return "Others"
    else:
        return x

In [297]:
df3['Item_Type'] = df3['Item_Type'].apply(others)

In [298]:
df3.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Years
0,Food,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,Small,Tier 3,Grocery Store,6.597664,22
1,Food,16.200,Regular,0.016687,Frozen Foods,96.9726,Small,Tier 2,Supermarket Type1,6.982490,18
2,Food,19.200,Regular,0.094450,Frozen Foods,187.8214,Small,Tier 2,Supermarket Type1,8.457769,13
3,Non-Consumable,13.000,Non-Edible,0.099887,Household,45.9060,Small,Tier 2,Supermarket Type1,6.733292,13
4,Food,5.925,Regular,0.161467,Dairy,45.5086,Small,Tier 3,Grocery Store,5.189810,22


In [299]:
df3['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Others                    956
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Name: Item_Type, dtype: int64

In [300]:
df_item_identifier = pd.get_dummies(df3['Item_Identifier'])
df_fat = pd.get_dummies(df3['Item_Fat_Content'])
df_outlet_size = pd.get_dummies(df3['Outlet_Size'])
df_outlet_location = pd.get_dummies(df3['Outlet_Location_Type'])
df_outlet_type = pd.get_dummies(df3['Outlet_Type'])
df_item_type = pd.get_dummies(df3['Item_Type'])

In [301]:
df4 = pd.concat([df3,df_fat,df_outlet_size,df_outlet_location,
                 df_outlet_type,df_item_type,df_item_identifier],axis='columns')

In [302]:
df4.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,...,Fruits and Vegetables,Health and Hygiene,Household,Meat,Others,Snack Foods,Soft Drinks,Drink,Food,Non-Consumable
0,Food,19.200,Regular,0.066132,Fruits and Vegetables,182.0950,Small,Tier 3,Grocery Store,6.597664,...,1,0,0,0,0,0,0,0,1,0
1,Food,16.200,Regular,0.016687,Frozen Foods,96.9726,Small,Tier 2,Supermarket Type1,6.982490,...,0,0,0,0,0,0,0,0,1,0
2,Food,19.200,Regular,0.094450,Frozen Foods,187.8214,Small,Tier 2,Supermarket Type1,8.457769,...,0,0,0,0,0,0,0,0,1,0
3,Non-Consumable,13.000,Non-Edible,0.099887,Household,45.9060,Small,Tier 2,Supermarket Type1,6.733292,...,0,0,1,0,0,0,0,0,0,1
4,Food,5.925,Regular,0.161467,Dairy,45.5086,Small,Tier 3,Grocery Store,5.189810,...,0,0,0,0,0,0,0,0,1,0


In [303]:
df5 = df4.drop(['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Size','Outlet_Location_Type','Outlet_Type'],axis=1)

In [304]:
df5.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Years,Low Fat,Non-Edible,Regular,High,Medium,...,Fruits and Vegetables,Health and Hygiene,Household,Meat,Others,Snack Foods,Soft Drinks,Drink,Food,Non-Consumable
0,19.200,0.066132,182.0950,6.597664,22,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
1,16.200,0.016687,96.9726,6.982490,18,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,19.200,0.094450,187.8214,8.457769,13,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,13.000,0.099887,45.9060,6.733292,13,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,5.925,0.161467,45.5086,5.189810,22,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [305]:
df5.dtypes

Item_Weight              float64
Item_Visibility          float64
Item_MRP                 float64
Item_Outlet_Sales        float64
Years                      int64
Low Fat                    uint8
Non-Edible                 uint8
Regular                    uint8
High                       uint8
Medium                     uint8
Small                      uint8
Tier 1                     uint8
Tier 2                     uint8
Tier 3                     uint8
Grocery Store              uint8
Supermarket Type1          uint8
Supermarket Type2          uint8
Supermarket Type3          uint8
Baking Goods               uint8
Canned                     uint8
Dairy                      uint8
Frozen Foods               uint8
Fruits and Vegetables      uint8
Health and Hygiene         uint8
Household                  uint8
Meat                       uint8
Others                     uint8
Snack Foods                uint8
Soft Drinks                uint8
Drink                      uint8
Food      

In [306]:
X = df5.drop('Item_Outlet_Sales' , axis=1)
Y = df5['Item_Outlet_Sales']

In [307]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,train_size=0.8)

In [308]:
from sklearn.linear_model import LinearRegression
lmodel = LinearRegression(normalize=True)

In [309]:
lmodel.fit(xtrain,ytrain)

LinearRegression(normalize=True)

In [310]:
ypred_train = lmodel.predict(xtrain)
ypred_test = lmodel.predict(xtest)

In [311]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
print(mean_absolute_error(ytrain,ypred_train))
print(mean_absolute_error(ytest,ypred_test))

0.4174165928823295
0.412148457334016


In [312]:
df3.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales', 'Years'],
      dtype='object')

In [313]:
def predict_sales(Item_Weight,Item_Visibility,Item_MRP,Years,Item_Fat_Content,Item_Type,
                 Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Identifier):

    input = np.zeros(xtrain.shape[1])
    input[0] = Item_Weight
    input[1] = Item_Visibility
    input[2] = Item_MRP
    input[3] = Years

    input[np.where(xtrain.columns == Item_Fat_Content)[0][0]] = 1
    input[np.where(xtrain.columns == Item_Type)[0][0]] = 1
    input[np.where(xtrain.columns == Outlet_Size)[0][0]] = 1
    input[np.where(xtrain.columns == Outlet_Location_Type)[0][0]] = 1
    input[np.where(xtrain.columns == Outlet_Type)[0][0]] = 1
    input[np.where(xtrain.columns == Item_Identifier)[0][0]] = 1

    return lmodel.predict([input])

## PREDICTION

In [314]:
print(predict_sales(9.30,0.03,250.20,20,
                    'Low Fat','Dairy','Medium','Tier 1','Supermarket Type2','Food'))

[8.25800184]


In [318]:
print(predict_sales(10.6,0.09,350.5,23,'Regular','Frozen Foods','Medium','Tier 3','Grocery Store','Non-Consumable'))

[7.27526818]
